## Projeto de Machine Learning para predicao do gênero literário de um livro a partir da sinopse




O projeto consiste em criar um modelo de machine learning que possa prever o gênero de um livro a partir da sinopse disponibilizada. Para isso sao comparados os algoritmos KNN, Árvore de Classificação, Naive Bayes e SVM para chegar ao melhor modelo, que entao pode ser utilizado para previsao de gêneros literários pelo utilizador.

## Configuração: Importação das bibliotecas necessárias.

Neste primeiro passo são importadas todas as bibliotecas necessárias para a configuracao, treinamento e selecao do melhor modelo.


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import re
import joblib

## Carga de Dados

Nesse passo os dados são exportados do dataset e são apresentadas algumas informações, como números de linhas e colunas a serem usados. Aqui é indicado o caminho para o dataset que vai ser usado para treinamento do modelo. Também sao especificados quais colunas e informacoes serao usadas no modelo.

In [ ]:
#  Configuração e Carregamento do Dataset 

print("Configuração e Carregamento do Dataset ")
df = pd.read_csv('../dataset/data.csv')
print(f"Número total de linhas do Dataset: {len(df)}")
print("\nPrimeiras 5 linhas do dataset:")
print(df.head())


# Definindo as colunas e os gêneros a serem utilizados
synopsis_column = 'summary'
genre_column = 'genre'
target_genres = ['horror', 'history', 'science', 'fantasy', 'thriller']

--- Configuração e Carregamento do Dataset ---
Número total de linhas do Dataset: 2998

Primeiras 5 linhas do dataset:
   index                      title    genre  \
0      0          Drowned Wednesday  fantasy   
1      1              The Lost Hero  fantasy   
2      2  The Eyes of the Overworld  fantasy   
3      3            Magic's Promise  fantasy   
4      4             Taran Wanderer  fantasy   

                                             summary  
0   Drowned Wednesday is the first Trustee among ...  
1   As the book opens, Jason awakens on a school ...  
2   Cugel is easily persuaded by the merchant Fia...  
3   The book opens with Herald-Mage Vanyel return...  
4   Taran and Gurgi have returned to Caer Dallben...  


## Filtragem dos dados e Separação entre treino e teste (holdout)

Nesse passo é feita a filtragem dos dados. Sao excluidas as linhas vazias, convesao do texto para minusculo, remocao de caracteres especiais e espacos e feita contagem e divisao do dataset em dados de treino e de teste. Para o modelo os dados de teste foram definidos em 20% e os dados de treino em 80%.

In [ ]:
#  Pré-processamento e Filtragem dos Dados 
print("\n Pré-processamento e Filtragem dos Dados ")

df_filtered = df[df[genre_column].isin(target_genres)].copy()
df_filtered.dropna(subset=[synopsis_column, genre_column], inplace=True)
df_filtered = df_filtered[df_filtered[synopsis_column].str.strip() != '']

print(f"Dataset filtrado para os gêneros {target_genres}.")
print(f"Número de linhas após filtragem e remoção de vazios: {len(df_filtered)}")
print("\nContagem de exemplos por gênero no dataset filtrado:")
print(df_filtered[genre_column].value_counts())

# Função de limpeza de texto (converte texto para minúsculas, remove caracteres
#que não são letras, números e remove espaços desnecessários)

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


df_filtered[synopsis_column] = df_filtered[synopsis_column].apply(clean_text)

# Divisão do Dataset 

print("\n Divisão do Dataset ")

X = df_filtered[synopsis_column]
y = df_filtered[genre_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Tamanho do conjunto de treino: {len(X_train)} amostras")
print(f"Tamanho do conjunto de teste: {len(X_test)} amostras")




--- Pré-processamento e Filtragem dos Dados ---
Dataset filtrado para os gêneros ['horror', 'history', 'science', 'fantasy', 'thriller'].
Número de linhas após filtragem e remoção de vazios: 2499

Contagem de exemplos por gênero no dataset filtrado:
genre
fantasy     500
science     500
history     500
thriller    500
horror      499
Name: count, dtype: int64

--- Divisão do Dataset ---
Tamanho do conjunto de treino: 1999 amostras
Tamanho do conjunto de teste: 500 amostras


## Transformação de dados: 

Modelos de Machine Learning em regra trabalham apenas com números e nao letras. Por isto nesse passo é feita a transformacao das letras em numeros atravez da vetorizacao TF-IDF, onde as numeros atribuidos a cada palavra representam a importancia deles no referido texto. A vetorizacao deve ser realizada tanto para o modelo de treino quanto para o de teste, e ao final o modelo é exportado para utilizacao nas entradas dos usuarios.

In [ ]:
# Representação de Texto (TF-IDF)

#Funcao que transforma o texto em numeros. 
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')


X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("\nTexto transformado em representacao numerica.")


Texto transformado em representacao numerica.


## Definicao dos algorítimos e parametros (utilização dos algoritmos KNN, Árvore de Classificação, Naive Bayes e SVM)

Neste passo definimos os modelos de classificacao a serem usados (KNN, Arvore de Classificacao, Naive Bayes e SVM). Tambem sao definidos os hiperparametros para otimizacao de cada modelo.

In [ ]:
#  Modelagem: Definição dos Algoritmos 


models_to_optimize = {
    'Naive Bayes': MultinomialNB(),
    'SVM': SVC(random_state=42),
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
}


param_grids = {
    'Naive Bayes': {
        'alpha': [0.1, 0.5, 1.0, 2.0]
    },
    'SVM': {
        'kernel': ['linear', 'rbf'],
        'C': [0.1, 1, 10]
    },
    'KNN': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance']
    },
    'Decision Tree': {
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10]
    }
}

print("Modelos e seus respectivos parâmetros para otimização definidos.")


Modelos e seus respectivos parâmetros para otimização definidos.


## Otimização de hiperparâmetros

Neste passo é realizada a otimizacao dos hiperparametros definidos anteriormente. A ideia aqui é encontrar a melhor combinacao dos hiperparamentos para garantir o melhor funcionamento possivel do modelo.

In [ ]:
#  Otimização de Hiperparâmetros com GridSearchCV 
print("\n Otimização de Hiperparâmetros com GridSearchCV ")

optimized_models = {} 

for name, model in models_to_optimize.items():
    print(f"\nIniciando otimização para {name}...")

   
    grid_search = GridSearchCV(model, param_grids[name], cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
    grid_search.fit(X_train_tfidf, y_train)

    optimized_models[name] = grid_search.best_estimator_ # Armazena o modelo com os melhores hiperparâmetros

    print(f"Otimização para {name} concluída.")
    print(f"Melhores Parâmetros para {name}: {grid_search.best_params_}")
    print(f"Melhor pontuação (CV Score) no treino para {name}: {grid_search.best_score_:.4f}")



--- Otimização de Hiperparâmetros com GridSearchCV ---

Iniciando otimização para Naive Bayes...
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Otimização para Naive Bayes concluída.
Melhores Parâmetros para Naive Bayes: {'alpha': 0.5}
Melhor pontuação (CV Score) no treino para Naive Bayes: 0.7399

Iniciando otimização para SVM...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Otimização para SVM concluída.
Melhores Parâmetros para SVM: {'C': 10, 'kernel': 'rbf'}
Melhor pontuação (CV Score) no treino para SVM: 0.7193

Iniciando otimização para KNN...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Otimização para KNN concluída.
Melhores Parâmetros para KNN: {'n_neighbors': 9, 'weights': 'distance'}
Melhor pontuação (CV Score) no treino para KNN: 0.5488

Iniciando otimização para Decision Tree...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Otimização para Decision Tree concluída.
Melhores Parâmetros para Decision Tree: {'max_depth'

## Avaliação e comparação de resultados dos modelos treinados com os diferentes algoritmos

Agora é criada uma matriz que mostra visualmente os acertos e erros. As linhas representam os gêneros verdadeiros, e as colunas, os gêneros previstos. É ótimo para identificar quais gêneros seu modelo está confundindo.

Seleção do Melhor Modelo: A lógica dentro do loop compara a acurácia de cada modelo no conjunto de teste. O modelo com a maior acurácia é identificado como best_model_name, e sua instância é armazenada em best_trained_model.

In [ ]:
#  Avaliação e Comparação dos Modelos Otimizados 
print("\n Avaliação e Comparação dos Modelos Otimizados ")


results = {}
best_model_name = None
best_accuracy = 0
best_trained_model = None

for name, model in optimized_models.items():
    y_pred = model.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)

    results[name] = {
        'accuracy': accuracy,
        'report': classification_report(y_test, y_pred, zero_division=0),
        'confusion_matrix': confusion_matrix(y_test, y_pred)
    }

    print(f"\n--- Resultados para o Modelo Otimizado: {name} ---")
    print(f"Acurácia no conjunto de teste: {accuracy:.4f}")
    print("Relatório de Classificação:\n", results[name]['report'])
    print("Matriz de Confusão:\n", results[name]['confusion_matrix'])

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model_name = name
        best_trained_model = model 

print("\n--- Comparação Final de Acurácia no Teste ---")
for name, res in results.items():
    print(f"{name}: Acurácia = {res['accuracy']:.4f}")

print(f"\nO modelo com melhor desempenho geral no conjunto de teste é: **{best_model_name}** (Acurácia: {best_accuracy:.4f})")



--- Avaliação e Comparação dos Modelos Otimizados ---

--- Resultados para o Modelo Otimizado: Naive Bayes ---
Acurácia no conjunto de teste: 0.7360
Relatório de Classificação:
               precision    recall  f1-score   support

     fantasy       0.76      0.68      0.72       100
     history       0.74      0.80      0.77       100
      horror       0.71      0.72      0.71       100
     science       0.71      0.77      0.74       100
    thriller       0.77      0.71      0.74       100

    accuracy                           0.74       500
   macro avg       0.74      0.74      0.74       500
weighted avg       0.74      0.74      0.74       500

Matriz de Confusão:
 [[68  7 10 11  4]
 [ 3 80  5  7  5]
 [10  6 72  5  7]
 [ 5  6  7 77  5]
 [ 4  9  8  8 71]]

--- Resultados para o Modelo Otimizado: SVM ---
Acurácia no conjunto de teste: 0.7380
Relatório de Classificação:
               precision    recall  f1-score   support

     fantasy       0.74      0.75      0.75      

## Exportação do modelo


Aqui estamos exportando o modelo para que possamos utilizar na nossa API. Os modelos sera exportados no formato .joblib apos a execucao dos testes e definicao do melhor modelo/algoritimo.

In [8]:
# ---  Exportar o Melhor Modelo Escolhido ---
print("\n--- Exportação do Melhor Modelo ---")

if best_trained_model is not None:
    model_filename = f'best_genre_classifier_{best_model_name.replace(" ", "_").lower()}.joblib'
    vectorizer_filename = 'tfidf_vectorizer.joblib'

    joblib.dump(best_trained_model, model_filename)
    joblib.dump(tfidf_vectorizer, vectorizer_filename)

    print(f"Modelo '{best_model_name}' exportado como '{model_filename}'")
    print(f"Vetorizador TF-IDF exportado como '{vectorizer_filename}'")



--- Exportação do Melhor Modelo ---
Modelo 'SVM' exportado como 'best_genre_classifier_svm.joblib'
Vetorizador TF-IDF exportado como 'tfidf_vectorizer.joblib'


## Exemplo de uso do modelo escolhido

Aqui sao informados dados aleatorios para o modelo exportado para podermos testar se o mesmo esta funcionando corretamente.

In [ ]:
#  Exemplo de Como Usar o Modelo Exportado 


print("\n Exemplo de Uso do Modelo Exportado ")
if best_trained_model is not None:
    loaded_model = joblib.load(model_filename)
    loaded_vectorizer = joblib.load(vectorizer_filename)

    test_synopsis = "A young officer awaits a mythical invasion in a desolate fort, his life consumed by futile expectation."
    cleaned_test_synopsis = clean_text(test_synopsis)
    test_synopsis_tfidf = loaded_vectorizer.transform([cleaned_test_synopsis])
    predicted_genre_loaded = loaded_model.predict(test_synopsis_tfidf)

    print(f"\nSinopse de teste: '{test_synopsis}'")
    print(f"Gênero previsto pelo modelo exportado: {predicted_genre_loaded[0]}")


--- Exemplo de Uso do Modelo Exportado ---

Sinopse de teste: 'A young officer awaits a mythical invasion in a desolate fort, his life consumed by futile expectation.'
Gênero previsto pelo modelo exportado: thriller


## Conclusao

Neste documento foram apresentados os passos para parametrizacao, teste e definicao de um modelo de machine learning, com boa aplicacao para o tipo de problema apresentado no projeto. Foram feitos testes para cada algoritimo e escolhido aquele que melhor se adequa. A partir da análise do resultado podemos concluir que apresenta resultado satisfatorio, sendo que nos testes realizados o resultado foi na maioria das vezes positivo.

Apesar de neste projeto em específico não ter sido dado ênfase as praticas de seguranca da informacao e desenvolvimento de software seguro, estes são extremamente importantes num contexto em que o mundo atual é cada vez mais digital e as inforcamoes sensiveis das pessoas estao disponiveis em quantidades enormes de bancos de dados na internet, muitas vezes resguardados sem a seguranca necessaria.

Este projeto em especifico nao utiliza dados sensiveis, apesar de nao sabermos a origem dos dados do dataset, extraido em site de acesso publico. Em analise inicial nao foram localizados dados que nao referentes a livro publicados e disponiveis a qualquer pessoa.